<a href="https://colab.research.google.com/github/obete/ClassifierDINOv3/blob/main/ClassifierDINOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **DINOv3 as base with a classifier head**


In [ ]:
from torchvision import transforms

# For ViT-Small/Base: 224x224, for ViT-Large: 384x384
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dir = f'{DRIVE_ROOT}/train_set'
val_dir = f'{DRIVE_ROOT}/val_set'

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Class names
print(train_dataset.classes)


In [ ]:
#@title Using DINOv3

!pip install -U transformers

Local Inference on GPU
Model page: https://huggingface.co/facebook/dinov3-vith16plus-pretrain-lvd1689m

⚠️ If the generated code snippets do not work, please open an issue on either the model repo and/or on huggingface.js 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# @title Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vith16plus-pretrain-lvd1689m")
backbone = AutoModel.from_pretrained("facebook/dinov3-vith16plus-pretrain-lvd1689m")

In [ ]:
#@title Freeze backbone → use DINOv3 as a fixed feature extractor.
for param in backbone.parameters():
    param.requires_grad = False  # freeze backbone


In [ ]:
#@title Custom Classification head
import torch.nn as nn

class DinoClassifier(nn.Module):
    def __init__(self, model, num_classes):
        super().__init__()
        self.model = model
        self.head = nn.Linear(model.config.hidden_size, num_classes)  # simple linear head

    def forward(self, x):
        features = self.model(x).last_hidden_state[:,0]  # CLS token
        return self.head(features)


In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = DinoClassifier(backbone, num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):  # Example: 5 epochs
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


In [ ]:
import torchmetrics

device = "cuda" if torch.cuda.is_available() else "cpu"

# Define metric objects
train_acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)
val_acc_metric   = torchmetrics.Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    train_loss = 0
    train_acc_metric.reset()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        train_acc_metric.update(outputs, labels)

    train_loss /= len(train_dataset)
    train_acc = train_acc_metric.compute()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_acc_metric.reset()
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            val_acc_metric.update(outputs, labels)

    val_loss /= len(val_dataset)
    val_acc = val_acc_metric.compute()

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"| Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
